In [2]:
# Uncomment to install dependencies
#!pip install pandas
#!pip install youtube_dl
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Requirement already up-to-date: google-api-python-client in /anaconda3/lib/python3.6/site-packages (1.7.11)
Requirement already up-to-date: google-auth-httplib2 in /anaconda3/lib/python3.6/site-packages (0.0.3)
Requirement already up-to-date: google-auth-oauthlib in /anaconda3/lib/python3.6/site-packages (0.4.1)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import pandas as pd
import random
import json
from youtube_dl import YoutubeDL

download_dir = "outputs"

In [5]:
trailers = pd.read_csv("trailers.csv", index_col=0)
#trailers

In [6]:
trailer_yt_links = 'https://www.youtube.com/watch?v=' + trailers['youtube_key']
#trailer_yt_links

In [7]:
ydl_opts = {
    'outtmpl': download_dir + '/%(title)s.%(ext)s',
    'format': 'bestvideo'
}
num_trailers_max = 100
with YoutubeDL(ydl_opts) as ydl:
    for t in trailer_yt_links[:num_trailers_max]:
        a = ydl.download([t])
        display(a)


[youtube] K_tLp7T6U1c: Downloading webpage
[youtube] K_tLp7T6U1c: Downloading embed webpage
[youtube] K_tLp7T6U1c: Refetching age-gated info webpage
[download] outputs/The Shawshank Redemption - Trailer.mp4 has already been downloaded
[download] 100% of 9.62MiB


0

[youtube] uMwECVb1q54: Downloading webpage
[youtube] uMwECVb1q54: Downloading embed webpage
[youtube] uMwECVb1q54: Refetching age-gated info webpage
[download] outputs/The Shawshank Redemption - Trailer.mp4 has already been downloaded
[download] 100% of 9.62MiB


0

[youtube] BXUEUwwgIyU: Downloading webpage
[youtube] BXUEUwwgIyU: Downloading video info webpage
[youtube] BXUEUwwgIyU: Downloading MPD manifest
[download] outputs/The Shawshank Redemption (1994) Re-Release Trailer.mp4 has already been downloaded
[download] 100% of 37.99MiB


0

[youtube] EsHlXpbMPOc: Downloading webpage
[youtube] EsHlXpbMPOc: Downloading video info webpage
[youtube] EsHlXpbMPOc: Downloading MPD manifest
[download] outputs/The Shawshank Redemption (1994) — Remastered Original Trailer.mp4 has already been downloaded
[download] 100% of 42.07MiB


0

[youtube] _IqFJLdV13o: Downloading webpage
[youtube] _IqFJLdV13o: Downloading video info webpage
[download] Resuming download at byte 42902620
[download] Destination: outputs/The Godfather - Official® Trailer [HD].mp4
[download]  69.5% of 60.35MiB at  1.61MiB/s ETA 00:114

KeyboardInterrupt: 